In [1]:
import open3d as o3d
import numpy as np
import pandas
from timeit import default_timer as time
from data_extraction import dataExtract
from xenon_reconstruct_extract import reconstruct_extract
from xenon_reconstruct_extract import build_path
import tylmen_api_functions
import os
import requests

filepath = './Point Clouds/set 1/brandon slow.ply'
pcd = o3d.io.read_point_cloud(filepath)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
path = build_path()
plypath = os.path.join(path, 'pointcloud.ply')
meshpath = os.path.join(path, 'mesh.obj')
dfpath = os.path.join(path, 'measurements.csv')
abspath = os.path.abspath(path)
print(path + "\n" + plypath + "\n" + meshpath + "\n" + dfpath + "\n" + abspath)

In [ ]:
api_test_path = "C:/Users/night/Documents/current/Senior Design/bucktest.mp4"
dotmov = open(api_test_path, 'rb')
tylmen_api_functions.api_upload_mov(dotmov)
dotmov.close()

In [3]:

status_url = "http://3.145.59.0:8000/api/check_status/"
#tylmen_api_functions.api_get_sane_status(status_url)
status_resp = requests.get(status_url)
print(status_resp.text)

In [ ]:
api_output = tylmen_api_functions.api_download_pipe()

In [6]:
download_url = "http://3.145.59.0:8000/api/download_ply/"
down_resp = requests.get(download_url)

In [7]:
print(down_resp.text)

<!DOCTYPE html>
<html lang="en">
<head>
  <meta http-equiv="content-type" content="text/html; charset=utf-8">
  <title>Page not found at /api/download_ply/buck1.ply</title>
  <meta name="robots" content="NONE,NOARCHIVE">
  <style type="text/css">
    html * { padding:0; margin:0; }
    body * { padding:10px 20px; }
    body * * { padding:0; }
    body { font:small sans-serif; background:#eee; color:#000; }
    body>div { border-bottom:1px solid #ddd; }
    h1 { font-weight:normal; margin-bottom:.4em; }
    h1 span { font-size:60%; color:#666; font-weight:normal; }
    table { border:none; border-collapse: collapse; width:100%; }
    td, th { vertical-align:top; padding:2px 3px; }
    th { width:12em; text-align:right; color:#666; padding-right:.5em; }
    #info { background:#f6f6f6; }
    #info ol { margin: 0.5em 4em; }
    #info ol li { font-family: monospace; }
    #summary { background: #ffc; }
    #explanation { background:#eee; border-bottom: 0px none; }
    pre.exception_value { 

In [ ]:

print("Writing PointCloud to " + plypath)
ply = open(plypath, 'wb')
ply.write(api_output)
ply.close()
point_cloud = o3d.io.read_point_cloud(path)

In [ ]:
pcd = o3d.io.read_point_cloud('./Point Clouds/set 1/brandon slow.ply')
#point_cloud = pcd

In [ ]:
mesh, measure_frame = reconstruct_extract(point_cloud)
    #code to output the mesh and extracted data to a .obj and .csv
print("Writing mesh to " + meshpath)
o3d.io.write_triangle_mesh(meshpath, mesh)
print("Writing measurements to " + dfpath)
measure_frame.to_csv(dfpath)

In [ ]:

print("Details:")
#print("Elapsed: " + str(end-start) + " seconds")
print(point_cloud)
print(mesh)
print(measure_frame.to_markdown())
print("Files can be found at:")
print(abspath)